## Preprocessing

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("../datasets/census_income_dataset.csv")

In [3]:
data = data.drop(["fnlwgt","education","relationship"], axis=1)

In [4]:
data.isnull().sum()

age                  0
workclass         1836
education-num        0
marital-status       0
occupation        1843
race                 0
sex                  0
capital-gains        0
capital-loss         0
hours-per-week       0
native-country     583
target               0
dtype: int64

In [5]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()

features_to_convert = ["workclass","marital-status","occupation","race","sex","native-country","target"]
for i in features_to_convert:
    data[i] = enc.fit_transform(data[i].astype('str'))
data.head()

,age,workclass,education-num,marital-status,occupation,race,sex,capital-gains,capital-loss,hours-per-week,native-country,target
0,39,6,13,4,0,4,1,2174,0,40,38,0
1,50,5,13,2,3,4,1,0,0,13,38,0
2,38,3,9,0,5,4,1,0,0,40,38,0
3,53,3,7,2,5,2,1,0,0,40,38,0
4,28,3,13,2,9,2,0,0,0,40,4,0


In [6]:
outliers = {}
for i in range(data.shape[1]):
    min_t = data[data.columns[i]].mean() - (3 * data[data.columns[i]].std())
    max_t = data[data.columns[i]].mean() + (3 * data[data.columns[i]].std())
    count = 0
    for j in data[data.columns[i]]:
        if j < min_t or j > max_t:
            count += 1
    outliers[data.columns[i]] = [count,data.shape[0]-count]
print(outliers)

{'hours-per-week': [440, 32121], 'native-country': [1113, 31448], 'workclass': [0, 32561], 'capital-gains': [215, 32346], 'age': [121, 32440], 'occupation': [0, 32561], 'sex': [0, 32561], 'education-num': [219, 32342], 'capital-loss': [1470, 31091], 'target': [0, 32561], 'marital-status': [0, 32561], 'race': [1350, 31211]}


## Activity 1

In [7]:
X = data.drop("target", axis=1)
Y = data["target"]

In [8]:
from sklearn.model_selection import train_test_split
X_new, X_test, Y_new, Y_test = train_test_split(X, Y, test_size=0.1, random_state=101)
X_train, X_dev, Y_train, Y_dev = train_test_split(X_new, Y_new, test_size=0.12, random_state=101)
print(X_train.shape, X_dev.shape, X_test.shape, Y_train.shape, Y_dev.shape, Y_test.shape)

(25787, 11) (3517, 11) (3257, 11) (25787,) (3517,) (3257,)


In [9]:
from sklearn.naive_bayes import GaussianNB
model_NB = GaussianNB()
model_NB.fit(X_train, Y_train)

GaussianNB(priors=None)

## Activity 2

In [10]:
from sklearn.tree import DecisionTreeClassifier
model_tree = DecisionTreeClassifier(random_state=101)
model_tree.fit(X_train, Y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=101,
            splitter='best')

## Activity 3

In [11]:
from sklearn.svm import SVC
model_svm = SVC(random_state=101)
model_svm.fit(X_train, Y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=101, shrinking=True,
  tol=0.001, verbose=False)

## Error analysis

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
X_sets = [X_train, X_dev, X_test]
Y_sets = [Y_train, Y_dev, Y_test]

metrics = {"Naive Bayes":{"Accuracy":[],"Precision":[],"Recall":[]},"Decision Tree":{"Accuracy":[],"Precision":[],"Recall":[]},"SVM":{"Accuracy":[],"Precision":[],"Recall":[]}}

for i in range(0,len(X_sets)):
    
    pred_NB = model_NB.predict(X_sets[i])
    metrics["Naive Bayes"]["Accuracy"].append(accuracy_score(Y_sets[i], pred_NB))
    metrics["Naive Bayes"]["Precision"].append(precision_score(Y_sets[i], pred_NB))
    metrics["Naive Bayes"]["Recall"].append(recall_score(Y_sets[i], pred_NB))
    
    pred_tree = model_tree.predict(X_sets[i])
    metrics["Decision Tree"]["Accuracy"].append(accuracy_score(Y_sets[i], pred_tree))
    metrics["Decision Tree"]["Precision"].append(precision_score(Y_sets[i], pred_tree))
    metrics["Decision Tree"]["Recall"].append(recall_score(Y_sets[i], pred_tree))
    
    pred_svm = model_svm.predict(X_sets[i])
    metrics["SVM"]["Accuracy"].append(accuracy_score(Y_sets[i], pred_svm))
    metrics["SVM"]["Precision"].append(precision_score(Y_sets[i], pred_svm))
    metrics["SVM"]["Recall"].append(recall_score(Y_sets[i], pred_svm))

metrics

{'Decision Tree': {'Accuracy': [0.9778958389886376,
   0.814046061984646,
   0.828676696346331],
  'Precision': [0.9875561700656758, 0.6313253012048192, 0.6321381142098274],
  'Recall': [0.919832582099163, 0.6009174311926605, 0.6287978863936592]},
 'Naive Bayes': {'Accuracy': [0.7973397448326677,
   0.7910150696616435,
   0.8087196806877495],
  'Precision': [0.6696490020646937, 0.6797900262467191, 0.6903409090909091],
  'Recall': [0.31326464906632323, 0.29701834862385323, 0.32100396301188905]},
 'SVM': {'Accuracy': [0.9108077713576609,
   0.8086437304520898,
   0.8182376420018422],
  'Precision': [0.8945139169019766, 0.7177242888402626, 0.7047146401985112],
  'Recall': [0.7139407598197038, 0.3761467889908257, 0.3751651254953765]}}